# Introduction
This tutorial aims to show how a 8x8 array of DNA tiles based on the work of Tikhomirov et al, shown below, can be created in `small`.
![title](tile-array.png)
The tutorial is divided into 3 parts, each aimed at introducing some of `small`s capabilites and the key ideas behind them while laying down the code needed to create an array of DNA tiles.
In the first part we cover the basic syntax of `common lisp` so that even those unfamilar with the language may easily follow along. Along side this we introduce the DNA model that `small` uses to represent DNA, showcase some of the API functionality for creating representations of objects made of DNA and define our own such object to represent the scaffold of one of the four triangles that a single tile in the array is made of. 

The next section shows how `small` allows one to build nanostructures of ever increasing size. We learn how to geometrically manipulate the triangle we created in part 1 and how to connect four such triangles to create a full tile used in the 8x8 array. We show how we can associate key-value pairs to parts of DNA structures and how we can retrieve structures based on these key-value pairs. Additionally how to create staple strands to hold the created tile together is also shown.

Part 3 shows how these tiles are in turn used to create a DNA array and how staple strands can be extended to create arbitrary patterns on these arrays.


## Introduction  to Common Lisp
Those unfamiliar with Lisp might be confused by the syntax and the abundance of parenthesis. This is no reason for concern, in-fact it is what give the Lisp family of languages their incredible expressive power. Lisp code is comprised of s-expressions, which can either be lists or atoms. Lists are delimited by parentheses and can contain any number of whitespace-separated elements. Atoms are everything else, for example a number, a character or a user defined object. Code and data are represented by lists, hence the large amount of parenthesis.

Code to call a function is composed of a list, with the first list entry being the function name and the next entries being its arguments. E.g.


In [1]:
(+ 1 2)

3

When evaluating multiple lists the inner most lists are evaluated before the enclosing list. E.g;

In [2]:
(- (+ 2 3) (+ 1 2))

2


Lists can also contain data, say for example a list of values (1 2) or all the nucleotides in a strand of DNA. If we wanted to create a list with data we cannot simply type (1 2) as Lisp will try to interpret first value in the list as the function name, resulting in an illegal function call `ERROR`.

In [3]:
(1 2)


COMPILED-PROGRAM-ERROR: Execution of a form compiled with errors.
Form:
  (1 2)
Compile-time error:
  illegal function call

; in: 1 2
;     (1 2)

; 
; caught ERROR:
;   illegal function call

; 
; compilation unit finished
;   caught 1 ERROR condition
Execution of a form compiled with errors.
Form:
  (1 2)
Compile-time error:
  illegal function call
   [Condition of type SB-INT:COMPILED-PROGRAM-ERROR]

#<ENVIRONMENT {100313B4C3}>
   [Environment of thread #<THREAD "SHELL Thread" RUNNING {1004D147C3}>]


Backtrace:
 5: (SB-KERNEL::%SIGNAL Execution of a form compiled with errors.
Form:
  (1 2)
Compile-time error:
  illegal function call)
 6: (ERROR COMPILED-PROGRAM-ERROR MESSAGE illegal function call SOURCE (1 2))
 7: ((LAMBDA NIL))
 8: (SB-INT:SIMPLE-EVAL-IN-LEXENV (1 2) #<NULL-LEXENV>)
 9: (EVAL (1 2))
 10: (COMMON-LISP-JUPYTER::MY-EVAL (1 2))
 11: ((:METHOD JUPYTER:EVALUATE-CODE (COMMON-LISP-JUPYTER:KERNEL T)) #<unused argument> (1 2)
)
 12: (JUPYTER::HANDLE-EXECUTE-REQUEST #<KERNEL {10039D0FB3}> #<MESSAGE {1002E81293}>)
 13: (JUPYTER::HANDLE-SHELL-MESSAGE #<KERNEL {10039D0FB3}> #<MESSAGE {1002E81293}>)


To get around this we can use the `quote` function. It takes one argument and returns exactly that argument, without evaluating anything in it or trying to interpret first list entry as a function name.

In [5]:
(quote (+ 1 2))

(+ 1 2)

In [6]:
(quote (:i (+ 1 2) (+ 2 3)))

(:I (+ 1 2) (+ 2 3))

Since lists are so prolific in Lisp the syntax has a lot of syntactic sugar to deal with their manipulation. For example instead or writing quote we can just write ' before the argument we normally pass to quote.

In [7]:
'(:i (+ 1 2) (+ 2 3))

(:I (+ 1 2) (+ 2 3))

Notice how the lists containing the `+` are not evaluated. If we did want it to be evaluated we could use the function `list` which takes n arguments, evaluates them and returns the values results from evaluating them in a list of length n.


In [8]:
(list :i (+ 1 2) (+ 2 3) 10 "DNA nanotechnology is fun!")

(:I 3 5 10 "DNA nanotechnology is fun!")

Another syntactic sugar that we use is backquote, `` ```. We need to understand its use if we want to understand create-staple and find-obj-with-props. Just like the syntactic sugar for quote, `'`, in fact using it in place of quote yields the same result.


In [12]:
`((+ 1 2) (+ 2 3))

((+ 1 2) (+ 2 3))

Where its behavior of `` ` `` is different to that of `'` is when an s-expression in the list is preceded by a `,`. When this happens the expression is evaluated and placed in the resulting list, e.g.

In [13]:
`((+ 1 2) ,(+ 2 3))

((+ 1 2) 5)

# Introduction to `small`

First we need to tell Common Lisp that we to use of the `small` package to gain access to its functionality it provides. First we load the package using [quicklisp](https://www.quicklisp.org/beta/).


In [15]:
(ql:quickload :small)


To load "small":
  Load 1 ASDF system:
    small

; Loading "small"
...

(:SMALL)

In [ ]:
N

In [16]:
(in-package :small)

#<PACKAGE "SMALL">

## Creating constants
In this section we  introduce the constants defined in [Tikhomirov et al](https://doi.org/10.1038/nnano.2016.256) that implement the geometric model of DNA Square (below) that they use.
![DNA Tile](greg-tile.png)

See [pages 7-11 of the supplementary information](https://static-content.springer.com/esm/art%3A10.1038%2Fnnano.2016.256/MediaObjects/41565_2017_BFnnano2016256_MOESM28_ESM.pdf) for information on the geometric model. For convenience we repeat the definition of the constants;

![title](consts.png)


To do so we use the [`defparameter` function](http://clhs.lisp.se/Body/m_defpar.htm), which binds a value to a variable. The list items that follow are the arguments to the function. In this case the fist argument is the variable name and the second argument is value to bind to it. Lisp also the ability code to add documentation to itself and the third argument is a docstring that adds such documentation describing the introduced variable. The surrounding *'s in the name of the variable is a convention in Common Lisp for denoting global variables.

Lets define the constants;


In [17]:
(defparameter *i1*  11.3d0
  "The length of the shortest double helix in the square DNA tile in nanometers")
     
(defparameter *r*  11
  "the total number of rows with increasing length in each of the four isosceles right triangles composing the square DNA tile")

(defparameter *2r*  (* 2 *r*)
  "The total number of rows in each of the four isosceles right triangles composing the square DNA tile")

(defparameter *g* 1.42d0 "Distance between the center of the square to the central vertex of each of the four triangles in nanometers")


(defparameter *w* (+ (* 2 (+ *i1* *g*))
		     (* (+ *helix-diameter* *helix-spacing*)
			(- (* 2 *r*)
			   1)))
  "The length of the side of the square DNA tile in nanometers")

*I1*

*R*

*2R*

*G*

*W*

We see that the variables have been correctly introduced.

In [18]:
*w*

88.44d0

## Calculating Number of Base Pairs Per Helix
Next we define the `ai` function (from the extract of the supplementary info above) that calculates the number of base pairs in each row of the double helix. To define a function we use the [defun](http://clhs.lisp.se/Body/m_defun.htm) function. which as its first argument takes the function name, then a list of variable that the function accepts (in this case only the row number `i`) followed by an optional documentation string and the body of the function.


In [20]:
(defun ai (i)
  "Calculate the number of base pairs in the i-th of the a DNA triangle that makes up the DNA square"
  (if (or (> i *2r*) (< i 0))
      (error "~a is an invalid index for ai calculation. valid indices: [1, ~a]" i *2r*)
      (if (<= i *r*)
	  (round (/ (+ *i1*
		       (* (+ *helix-diameter* *helix-spacing*)
			  (- i 1)))
		    *helix-nt-spacing*))
	  (ai (+ *2r* 1 (- i))))))

AI

[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::AI in DEFUN


Notice how we use the global variables `*helix-diameter*` `*helix-spacing*` and `*helix-nt-spacing*`. These are variable provided by `small` (see the [dna.lisp](https://github.com/DurhamSmith/small/blob/master/dna.lisp) file) that defines common parameters of DNA.

Also since we added a docstring to the function we are able to retrieve this (and other) information about the function using the `describe` function.


In [21]:
(describe #'ai)


#<FUNCTION AI>
  [compiled function]


Lambda-list: (I)
Derived type: (FUNCTION (T) *)
Documentation:
  Calculate the number of base pairs in the i-th of the a DNA triangle that makes up the DNA square
Source form:
  (LAMBDA (I)
    "Calculate the number of base pairs in the i-th of the a DNA triangle that makes up the DNA square"
    (BLOCK AI
      (IF (OR (> I *2R*) (< I 0))
          (ERROR
           "~a is an invalid index for ai calculation. valid indices: [1, ~a]"
           I *2R*)
          (IF (<= I *R*)
              (ROUND
               (/
                (+ *I1*
                   (* (+ *HELIX-DIAMETER* *HELIX-SPACING*) (- I 1)))
                *HELIX-NT-SPACING*))
              (AI (+ *2R* 1 (- I)))))))


If we only wanted the documentation can use the `documentation` function.

In [23]:
(documentation #'ai t)

"Calculate the number of base pairs in the i-th of the a DNA triangle that makes up the DNA square"

Notice since we wanted information on a function we use the `#'` syntax, this means we want the information on the function contained in the symbol named `ai`. Had we not used this we would have run into an error since we would try to retrieve information on the variable (not the function).


In [24]:
(describe ai)


UNBOUND-VARIABLE: The variable AI is unbound.

The variable AI is unbound.
   [Condition of type UNBOUND-VARIABLE]

#<ENVIRONMENT {1006C20ED3}>
   [Environment of thread #<THREAD "SHELL Thread" RUNNING {1004D147C3}>]


Backtrace:
 5: (SB-KERNEL::%SIGNAL The variable AI is unbound.)
 6: (ERROR The variable AI is unbound.)
 7: (SB-KERNEL:WITH-SIMPLE-CONDITION-RESTARTS ERROR NIL The variable AI is unbound.)
 8: ((LABELS SB-KERNEL::TRY :IN SB-KERNEL::RESTART-UNBOUND) The variable AI is unbound.)
 9: (SB-KERNEL::RESTART-UNBOUND AI The variable AI is unbound. -20151)
 10: (SB-KERNEL:INTERNAL-ERROR #.(SB-SYS:INT-SAP #X7F0E15CA5A40) #<unused argument>)
 11: ("foreign function: call_into_lisp")
 12: ("foreign function: funcall2")
 13: ("foreign function: interrupt_internal_error")
 14: ("foreign function: handle_trap")
 15: ("foreign function: #x418812")
 16: (SB-INT:SIMPLE-EVAL-IN-LEXENV AI #<NULL-LEXENV>)
 17: (SB-INT:SIMPLE-EVAL-IN-LEXENV (DESCRIBE AI) #<NULL-LEXENV>)
 18: (EVAL (DESCRIBE AI))
 19: (COMMON-LISP-JUPYTER::MY-EVAL (DESCRIBE

In [ ]:
`small` is fully documented and the documentation can be viewed at anytime using `describe` and `documentation` or viewed 